In [1]:
import numpy as np
import math

In [2]:
class em_algo:
    
    def __init__(self, no_of_mov, no_of_ids, no_of_iter, own_id):
        self.no_of_mov = no_of_mov
        self.no_of_ids = no_of_ids
        self.no_of_iter = no_of_iter
        self.own_id = own_id
     
        
    def file_loader(self):
        self.stud_ids = np.loadtxt("hw8_ids.txt", dtype=str,delimiter="\n")
        self.mov_list = np.loadtxt("hw8_movies.txt", dtype=str)
        self.prob_R = np.loadtxt("hw8_probR_init.txt", dtype=float)
        self.prob_Z  = np.loadtxt("hw8_probZ_init.txt", dtype=float)
        self.rat = np.loadtxt("hw8_ratings.txt", dtype=str)
        
    
    def watch_func(self,array):
        ln = len(array)
        c = 0
        for i in range(ln):
            if array[i] != "?":
                c += 1
        return c

    def rec_func(self,array):
        ln = len(array)
        c = 0
        for i in range(ln):
            if array[i] == "1":
                c += 1
        return c

    
    def pop_func(self):
        
        m_pop = np.zeros((self.no_of_mov,2))
        for i in range(self.no_of_mov):
            m_pop[i][0] = self.rec_func(self.rat[:,i])/self.watch_func(self.rat[:,i])
            m_pop[i][1] = i
        m_pop = m_pop[np.argsort(m_pop[:, 0])]
        print('Answer for Part (a) \n')
        print('List of movies in increasing order of popularity:\n')
        for i in range(self.no_of_mov):
            print(self.mov_list[int(m_pop[i][1])])
                     
#################                                                                          #####################            
            
    def omega_t_func(self,R_var,t,i):
        p = 1.0
        for j in range(self.no_of_mov):
            if self.rat[t][j] == "1":
                p *= float(R_var[j][i])
            elif self.rat[t][j] == "0":
                p *= (1-float(R_var[j][i]))
        return p

    def log_likelihood_func(self,Z_var,R_var):
        s= 0.0
        for t in range(self.no_of_ids):
            s2 = 0.0
            for i in range(4):
                s2 += (float(Z_var[i]) * self.omega_t_func(R_var,t,i))
            s += np.log(s2)
        return s/self.no_of_ids

    def e_step_func(self,Z_var,R_var,t,i):
        nr = float(Z_var[i]) * float(self.omega_t_func(R_var,t,i))
        dr = (float(Z_var[0]) * float(self.omega_t_func(R_var,t,0))) + (float(Z_var[1]) * float(self.omega_t_func(R_var,t,1))) + (float(Z_var[2]) * float(self.omega_t_func(R_var,t,2))) + (float(Z_var[3]) * float(self.omega_t_func(R_var,t,3)))
        return nr/dr


    def m_step_r_func(self,row,i,R_var,j):
        n1 = 0
        n2 = 0
        s = 0
        for t in range(self.no_of_ids):
            s += row[i][t]
            if  self.rat[t][j] =="1":
                n1 += row[i][t]
            elif  self.rat[t][j] =="?" :
                n2 +=  row[i][t] * float(R_var[j][i])
        return (n1+n2)/s
    
    
    def m_step_z_func(self,row,i):
        s = 0
        for t in range(self.no_of_ids):
            s += row[i][t]
        z = s/self.no_of_ids
        return z

######################                                                                      ##################
            
    def itr_log_lik(self):
        Z_var_itr = np.zeros(4)
        Z_var_itr = self.prob_Z
        self.R_var_itr = np.zeros((self.no_of_mov,4))
        self.R_var_itr =  self.prob_R
        self.row_h = np.zeros((4,self.no_of_ids))
        print("Iteration \t Log-likelihood")
        print(0, self.log_likelihood_func(Z_var_itr,self.R_var_itr))
        for itr in range(self.no_of_iter):
         
            for i in range(4):
                for self.t in range(self.no_of_ids):
                    self.row_h[i][self.t] = self.e_step_func(Z_var_itr,self.R_var_itr,self.t,i)
          
            for i in range(4):
                Z_var_itr[i] = self.m_step_z_func(self.row_h,i)
                for j in range (self.no_of_mov):
                    self.R_var_itr[j][i] = self.m_step_r_func(self.row_h,i,self.R_var_itr,j)
            if itr+1 in [1,2,4,8,16,32,64,128,256]:
                print(itr+1, self.log_likelihood_func(Z_var_itr,self.R_var_itr))
                
    def implementation(self):
        id_row = int(np.where(self.stud_ids==self.own_id)[0])
        r = 0 
#         print(id_row)
        dict ={}
        for j in range(self.no_of_mov):
            if self.rat[id_row][j] == "?":
                r =0
                for i in range(4):
                    r = self.row_h[i][id_row]*self.R_var_itr[j][i]
                dict[self.mov_list[j]] = r
        print('Answer for Part (f) \n')
        print('List of unseen movies in increasing order of expected ratings:\n')
        final_list = sorted(dict.items(), key=lambda x: x[1],reverse=True)
        return final_list

        

        
            
    
        
        
        

In [3]:
class_call = em_algo(76,362,256,"A59011408")
class_call.file_loader()
class_call.pop_func()


Answer for Part (a) 

List of movies in increasing order of popularity:

I_Feel_Pretty
Fifty_Shades_of_Grey
Hustlers
The_Last_Airbender
Magic_Mike
Fast_&_Furious:_Hobbs_&_Shaw
The_Shape_of_Water
Prometheus
Phantom_Thread
World_War_Z
Star_Wars:_The_Force_Awakens
Rocketman
Chappaquidick
Bridemaids
Man_of_Steel
American_Hustle
Terminator:_Dark_Fate
Room
Good_Boys
Pokemon_Detective_Pikachu
Fast_Five
Mad_Max:_Fury_Road
Drive
Us
The_Help
Pitch_Perfect
Jurassic_World
Frozen
X-Men:_First_Class
The_Revenant
Ex_Machina
Avengers:_Age_of_Ultron
La_La_Land
Midnight_in_Paris
Manchester_by_the_Sea
Once_Upon_a_Time_in_Hollywood
Three_Billboards_Outside_Ebbing
Darkest_Hour
The_Great_Gatsby
Dunkirk
Her
Captain_America:_The_First_Avenger
The_Girls_with_the_Dragon_Tattoo
Ready_Player_One
Hidden_Figures
The_Hateful_Eight
Thor
Toy_Story_3
The_Hunger_Games
12_Years_a_Slave
Iron_Man_2
The_Perks_of_Being_a_Wallflower
Joker
Les_Miserables
21_Jump_Street
Spiderman:_Far_From_Home
Black_Swan
Parasite
The_Avengers


In [4]:
class_call.itr_log_lik()

Iteration 	 Log-likelihood
0 -27.03581500351123
1 -17.560403824314395
2 -16.002362630627825
4 -15.06059731789225
8 -14.501649272824999
16 -14.263788571437443
32 -14.180178075094313
64 -14.170077781591026
128 -14.163960358152176
256 -14.163692439007862


In [5]:
class_call.implementation()

Answer for Part (f) 

List of unseen movies in increasing order of expected ratings:



[('Manchester_by_the_Sea', 0.6193436741565695),
 ('Chappaquidick', 0.6187523676686918),
 ('Darkest_Hour', 0.6045864463443938),
 ('Gone_Girl', 0.601019332340447),
 ('Django_Unchained', 0.5884980413029929),
 ('Ready_Player_One', 0.5689803646974404),
 ('Les_Miserables', 0.5688655597988229),
 ('Phantom_Thread', 0.5659124093250973),
 ('Three_Billboards_Outside_Ebbing', 0.5623893613227958),
 ('The_Girls_with_the_Dragon_Tattoo', 0.554991134765305),
 ('The_Farewell', 0.5532668651517609),
 ('Hidden_Figures', 0.5483581009500175),
 ('Fast_Five', 0.5451290878778453),
 ('Black_Swan', 0.5404367210627946),
 ('Pitch_Perfect', 0.5385295979653715),
 ('Once_Upon_a_Time_in_Hollywood', 0.536786707220431),
 ('American_Hustle', 0.5363152624359652),
 ('La_La_Land', 0.5317296886626047),
 ('The_Perks_of_Being_a_Wallflower', 0.525792678779205),
 ('The_Help', 0.5227862116656057),
 ('Drive', 0.5154059981961385),
 ('Bridemaids', 0.5022222636039826),
 ('Good_Boys', 0.4913492754171423),
 ('The_Hateful_Eight', 0.48478